# Download dataset from Kaggle

In [1]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pwd
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download m5-forecasting-accuracy

! unzip /content/m5-forecasting-accuracy.zip > /dev/null

-rw-r--r-- 1 root root 74 Sep  6 13:59 kaggle.json
/content
 55% 25.0M/45.8M [00:00<00:00, 123MB/s]
100% 45.8M/45.8M [00:00<00:00, 152MB/s]


# Libraries

In [2]:
!pip uninstall xgboost
! pip install https://s3-us-west-2.amazonaws.com/xgboost-nightly-builds/release_2.0.0/xgboost-2.0.0rc1%2B06487d3896c7090cf0c849547f55f4e07d009b97-py3-none-manylinux2014_x86_64.whl > /dev/null

Found existing installation: xgboost 1.7.6
Uninstalling xgboost-1.7.6:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/xgboost-1.7.6.dist-info/*
    /usr/local/lib/python3.10/dist-packages/xgboost.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.10/dist-packages/xgboost/*
Proceed (Y/n)? y
  Successfully uninstalled xgboost-1.7.6


In [14]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.subplots as sp
import plotly.express as px
from tqdm import tqdm
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import warnings
import random
import joblib
import os
import gc

warnings.filterwarnings("ignore")
seed = 101
random.seed(seed)
np.random.seed(seed)

# Preparing dataset

In [2]:
sales = pd.read_csv('sales_train_evaluation.csv')
sales.name = 'sales'
calendar = pd.read_csv('calendar.csv')
calendar.name = 'calendar'
prices = pd.read_csv('sell_prices.csv')
prices.name = 'prices'
calendar.replace(np.nan, 'No_event', inplace=True)

In [3]:
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

In [4]:
categorical_columns = sales.select_dtypes(include=['object']).columns.tolist()
sales[categorical_columns] = sales[categorical_columns].astype('category')

In [7]:
sales.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We decided to use a technique called downcasting. It can help us to be more efficient.

In [5]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in tqdm(enumerate(types)):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

print('sales before downcast: ', sales.memory_usage().sum())
sales = downcast(sales)
print('sales after downcast: ', sales.memory_usage().sum())
print('prices before downcast: ', prices.memory_usage().sum())
prices = downcast(prices)
print('prices after downcast: ', prices.memory_usage().sum())
print('calendar before downcast: ', calendar.memory_usage().sum())
calendar = downcast(calendar)
print('calendar after downcast: ', calendar.memory_usage().sum())

sales before downcast:  476792416


1975it [03:04, 10.68it/s] 


sales after downcast:  101826396
prices before downcast:  218916000


4it [00:00,  4.03it/s]


prices after downcast:  47978835
calendar before downcast:  220656


14it [00:00, 991.91it/s]

calendar after downcast:  131456


### merge all data files

In [6]:
gc.collect()

0

In [7]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

In [8]:
df = pd.merge(df, calendar, on='d', how='left')

In [9]:
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

In [10]:
df.dropna(inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47735397 entries, 7 to 60034809
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            category      
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             object        
 7   sold          int16         
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       category      
 11  wday          int8          
 12  month         int8          
 13  year          int16         
 14  event_name_1  category      
 15  event_type_1  category      
 16  event_name_2  category      
 17  event_type_2  category      
 18  snap_CA       int8          
 19  snap_TX       int8          
 20  snap_WI       int8          
 21  sell_price    float16       
dtypes: category(11), datetime64[ns](1), float16(1), int16(3), int8(5

In [16]:
df.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,No_event,No_event,No_event,No_event,0,0,0,NaN


In [17]:
df.tail(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
60034809,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,...,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.0


In [12]:
df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sold', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'],
      dtype='object')

In [19]:
df.isnull().sum()

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
d                      0
sold                   0
date                   0
wm_yr_wk               0
weekday                0
wday                   0
month                  0
year                   0
event_name_1           0
event_type_1           0
event_name_2           0
event_type_2           0
snap_CA                0
snap_TX                0
snap_WI                0
sell_price      12299413
dtype: int64

In [13]:
d_id = dict(zip(df.id.cat.codes, df.id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))

df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes

df.drop('date',axis=1,inplace=True)

# Feature engineering

In [15]:
df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)

# Train

In [57]:
valid = df[(df['d']>=1914) & (df['d']<1942)][['id','d','sold']]
test = df[df['d']>=1942][['id','d','sold']]
eval_preds = test['sold']
valid_preds = valid['sold']

In [58]:
xgb_params = {
    'objective': 'reg:squarederror',
    'n_estimators': 1000,
    'learning_rate': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'max_depth': 8,
    'min_child_weight': 300,
    'eval_metric': 'rmse',
}


depts = sales.dept_id.cat.codes.unique().tolist()

for dept in depts:
    df_temp = df[df['dept_id'] == dept]
    model_filename = 'xgboost_model' + str(d_dept_id[dept]) + '.pkl'

    X_train, y_train = df_temp[df_temp['d']<1914].drop('sold',axis=1), df_temp[df_temp['d']<1914]['sold']
    X_valid, y_valid = df_temp[(df_temp['d']>=1914) & (df_temp['d']<1942)].drop('sold',axis=1), df_temp[(df_temp['d']>=1914) & (df_temp['d']<1942)]['sold']
    X_test = df_temp[df_temp['d']>=1942].drop('sold',axis=1)

    print('*****Prediction for dept of: {}*****'.format(d_dept_id[dept]))
    model = xgb.XGBRegressor(**xgb_params, tree_method='gpu_hist', gpu_id=0)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],
              early_stopping_rounds=20, verbose=20)

    valid_preds[X_valid.index] = model.predict(X_valid)
    eval_preds[X_test.index] = model.predict(X_test)

    joblib.dump(model, model_filename)

    del model, X_train, y_train, X_valid, y_valid, X_test, df_temp
    gc.collect()

*****Prediction for dept of: HOBBIES_1*****
[0]	validation_0-rmse:2.36157	validation_1-rmse:2.11226
[20]	validation_0-rmse:2.06412	validation_1-rmse:1.85999
[40]	validation_0-rmse:2.04475	validation_1-rmse:1.85147
[60]	validation_0-rmse:2.03289	validation_1-rmse:1.84610
[80]	validation_0-rmse:2.02458	validation_1-rmse:1.84119
[100]	validation_0-rmse:2.01797	validation_1-rmse:1.83875
[120]	validation_0-rmse:2.01155	validation_1-rmse:1.83724
[136]	validation_0-rmse:2.00755	validation_1-rmse:1.83968
*****Prediction for dept of: HOBBIES_2*****
[0]	validation_0-rmse:0.83012	validation_1-rmse:0.85895
[20]	validation_0-rmse:0.78951	validation_1-rmse:0.82470
[40]	validation_0-rmse:0.78401	validation_1-rmse:0.82150
[59]	validation_0-rmse:0.78078	validation_1-rmse:0.82201
*****Prediction for dept of: HOUSEHOLD_1*****
[0]	validation_0-rmse:2.69590	validation_1-rmse:2.39316
[20]	validation_0-rmse:2.10730	validation_1-rmse:1.93187
[40]	validation_0-rmse:2.05867	validation_1-rmse:1.89975
[60]	valida

In [ ]:
feature_importance = pd.DataFrame()
features = [f for f in df.columns if f != '10sold']
for filename in os.listdir('/content'):
    if 'model' in filename:
        model = joblib.load(filename)
        dept_importance = pd.DataFrame()
        dept_importance["feature"] = features
        dept_importance["importance"] = model.feature_importances_
        dept_importance["dept"] = filename[5:9]
        feature_importance = pd.concat([feature_importance, dept_importance], axis=0)

def display_importances(feature_importance):
    cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:20].index
    best_features = feature_importance.loc[feature_importance.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.tight_layout()

display_importances(feature_importance)

# Submission

In [60]:
validation = sales[['id']+['d_' + str(i) for i in range(1914,1942)]]
validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
validation['id'] = validation['id'].astype('string')
validation["id"] = validation["id"].apply(lambda x: x.replace("evaluation", "validation"))

test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.head(1)

d,id,1942,1943,1944,1945,1946,1947,1948,1949,1950,...,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
0,0,0.790389,0.790389,0.790389,0.790389,0.861177,1.00621,0.91378,0.790389,0.790389,...,0.861177,1.022752,0.930323,0.803734,0.816261,0.829605,0.790389,0.861177,1.00621,0.91378


In [61]:
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
evaluation.id = evaluation.id.map(d_id)

submit = pd.concat([validation,evaluation]).reset_index(drop=True)
submit.to_csv('submission.csv',index=False)

In [62]:
submit.head(1)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0


In [63]:
submit.tail(1)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60979,HOUSEHOLD_2_516_WI_3_evaluation,0.138294,0.115373,0.130745,0.149506,0.171394,0.173609,0.167303,0.138559,0.115373,...,0.172154,0.177907,0.171601,0.14418,0.117424,0.136039,0.151557,0.173444,0.175659,0.154053


In [64]:
! kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m "dept loop, native, FEs, early stop 20, lr: 0.3"

100% 13.4M/13.4M [00:02<00:00, 5.48MB/s]
Successfully submitted to M5 Forecasting - Accuracy